In [1]:
#Import of libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
# Configuration

#Activate interactive shell
%config InteractiveShell.ast_node_interactivity = 'all'

In [3]:
# Import the csv file that holds the exported data from simulating PID controller in OpenModelica
dfOpenModPID = pd.read_csv('datafiles/exportedPIDsimulationFromOpenModelica.csv')

# Import the csv file that holds the simulation data from simulating PID controller in Python with FMU
dfFMUpythonPID = pd.read_csv("datafiles/airHeaterPIDsimulation.csv")

In [4]:
# Remove the percentiles from the output and display stats
perc = []
dfOpenModPID.describe(percentiles=perc)
dfFMUpythonPID.describe(percentiles=perc)

,time,airHeaterTextWithIO.T_Out_ext,airHeaterTextWithIO.T_amb_ext,airHeaterTextWithIO.u_ext,Reference.y
count,508.000000,508.000000,508.000000,508.000000,508.000000
mean,299.027559,23.242521,18.334646,1.492704,23.326772
std,173.708010,2.400128,0.747264,0.987310,2.361658
min,0.000000,19.635592,18.000000,0.000000,20.000000
50%,297.000000,25.000000,18.000000,1.999982,25.000000
max,600.000000,25.934950,20.000000,5.000000,25.000000


,# time,controlSignal,outputTemperature
count,600001.000000,600001.000000,600001.000000
mean,300.001000,1.485131,23.304572
std,173.205514,0.975682,2.411911
min,0.001000,0.000000,19.632972
50%,300.001000,1.999989,25.000000
max,600.001000,5.000000,26.926412
